In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
path = os.getcwd()
print(path)

/Users/jan/Dropbox/Python_Scripts/ags_trans


Read in files

In [3]:
files = ['ref-gemeinden-1990-2000.xlsx', 'ref-gemeinden-2000-2010.xlsx', 'ref-gemeinden-2010-2020.xlsx']
dfs = {}
for year in range(1990, 2000):
    dfs[f'{year}_{year+1}'] = pd.read_excel(f'{path}/data/{files[0]}', sheet_name=f'{year}-{year+1}', header=1, decimal = ',')
for year in range(2000, 2010):
    dfs[f'{year}_{year+1}'] = pd.read_excel(f'{path}/data/{files[1]}', sheet_name=f'{year}-{year+1}', header=1, decimal = ',')
for year in range(2010, 2020):
    if year < 2016:
        dfs[f'{year}_{year+1}'] = pd.read_excel(f'{path}/data/{files[2]}', sheet_name=f'{year}-{year+1}', header=1, decimal = ',')
    else: # header changed
        dfs[f'{year}_{year+1}'] = pd.read_excel(f'{path}/data/{files[2]}', sheet_name=f'{year}-{year+1}', header=0, decimal = ',')

Keep only the Kennziffer (AGS) columns (and the name for 1990 and 2020)

In [4]:
AGS_dfs = {}
for name, df in dfs.items():
    if name == '1990_1991':
        AGS_dfs[name] = dfs[name][['Name', 'Kennziffer', 'Kennziffer.1']] #keep name 1990
    elif name == '2019_2020':
        AGS_dfs[name] = dfs[name][['Gemeindename 2020', 'Kennziffer', 'Kennziffer.1']] #keep name 2020
    else:
        AGS_dfs[name] = dfs[name][['Kennziffer', 'Kennziffer.1']]

Rename the AGS columns

In [5]:
for name, df in AGS_dfs.items():
    old, new = name.split('_')
    if name == '1990_1991':
        col_names = ['Name_1990', f'AGS_{old}', f'AGS_{new}']
    elif name == '2019_2020':
        col_names = ['Name_2020', f'AGS_{old}', f'AGS_{new}']
    else:
        col_names = [f'AGS_{old}', f'AGS_{new}']
    df.columns = col_names
    if name == '1990_1991':
        df['Name_1990'] = df['Name_1990'].str.title()

/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T/ipykernel_77858/1259977040.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Name_1990'] = df['Name_1990'].str.title()


Merge all dataframes

In [6]:
AGS_trans = AGS_dfs['1990_1991']
for year in range(1990, 2019):
    AGS_trans = AGS_trans.merge(AGS_dfs[f'{year+1}_{year+2}'], on=f'AGS_{year+1}')

Rearrange

In [7]:
column_to_move = AGS_trans.pop("Name_2020")
AGS_trans.insert(1, "Name_2020", column_to_move)

Convert AGS to string and make 8 digits long

In [8]:
for year in range(1990, 2021):
    AGS_trans[f'AGS_{year}'] = AGS_trans[f'AGS_{year}'].astype(str)
    AGS_trans[f'AGS_{year}'] = np.where(AGS_trans[f'AGS_{year}'].str.len() < 8, '0' + AGS_trans[f'AGS_{year}'], AGS_trans[f'AGS_{year}'])

Keep only the obs where the AGS was changed

In [9]:
AGS_changes = AGS_trans[AGS_trans.filter(like='AGS').nunique(axis=1).gt(1)]

Export dfs

In [10]:
AGS_trans = AGS_trans.set_index('Name_1990')
AGS_changes = AGS_changes.set_index('Name_1990')

In [11]:
AGS_trans.to_csv(f'{path}/data/ags_transition.csv', encoding = 'utf-8-sig')
AGS_changes.to_csv(f'{path}/data/ags_changes.csv', encoding = 'utf-8-sig')